# **Stock Market Analysis for Tech Stocks**



**1. Data Source and Scope of Data:**
We had begun with a dataset containing historical daily prices for only 1 year for all tickers (companies) in **Mega, Large, Mid and Small cap** currently trading on S&P 500(stock prices of the 500 largest publicly traded companies in the United States).

Realizing that,
1. At least 5-10 years of data is needed to find any trends and seasonality patterns in the
behavior of the stock, and
2. for now, assuming that the value of one stock is independent from the value and behavior of stock of any other ticker in the same cap and category,

We have limited the dataset to contain data for only **company: Microsoft** for our EDA and modelling for now. To be able to find the trends and seasonality patterns we have increased the scope of data to the last 10 years. The same EDA and modelling techniques can be applied to other Technology category stocks and used for a comparison and recommendation system as mentioned in our initial proposal.


## **2. Strategy:**
Since stocks data is a time series, we are using a combination of **data engineering, feature engineering, machine learning, and hyperparameter optimization techniques.**
1. Data collection and exploration
2. Data preprocessing
3. Feature Engineering, augmentation, and selection
4. Model Performance evaluation and improvement
5. Model comparison


## **3. Data Source**:

We collected the data from YahooFinance using yfinance python library for ticker ‘MSFT’.

**Data Definition**:


**Columns and Description**

---


* Symbol:     Symbol using which a company is represented. Since we are only doing our analysis on Microsoft, the value is “MSFT”
* Date:       This column typically represents the date and/or time of the data point.Our data spans from 1st March, 2014 to 31st March, 2024.
* Open:       The opening price of the stock for the given date/time.
* Low:        The lowest price the stock reached during the trading day.
* High:       The highest price the stock reached during the trading day.
* Close:      The closing price of the stock for the given date/time.
* Adj Close:  The adjusted closing price, which accounts for corporate actions such as dividends, stock splits, and mergers.
* Volume:     The number of shares traded during the trading day.


## **4. EDA and Data Preprocessing:**

* **1) Missing Values:** No missing values were found.
* **2) Data Types:** The date column was an object. We changed the datatype to date with format ‘%Y-%m-%d’.
* **3) Outliers:** No outliers were found in the data as depicted in the time series plot below. We will use “Close” column for our analysis.
* **4) Checking for duplicates:** No duplicates were found.


## **5. Feature Engineering:**
We have added features in the dataset for various periods of time: 14, 30, 50 and 200 days.

* **a. Moving average:** We are using exponential moving average.
* **b. Relative Strength Index or RSI:** Used to assess the strength or weakness of a stock's price movements over a specified period.
* **c. 1 day volume change:** Percent change of volume from the previous 1 day.
* **d. 7-day future close:** Future close price of the stock by shifting the 'Adj Close' column 7 rows (days) into the future using .shift(-7).
* **e. 7-day future close percentage change (7d_close_future_pct):** percentage change in the future close price ('7d_future_close') compared to the close price 7 days ago.


## **6. Models:**

* **1) Random Forest:**

Target Variable: 7d_close_future_pct
Goal: To predict the % change in the price of the stock 7 days ahead into the future.
Results: The test score gave a negative number for R-squared. This indicates that the model performs worse than a horizontal regression line fitted to the data.
Bagging model performs slightly better but is still not robust enough.
* **2) LSTM:**
Target Variable: Close (closing price for the given day)
Goal: To predict the closing price for the given day looking at past 60 days)
We scaled the data and used a sequence length of 60 i.e., used 60 days data to process at once.
* **3) Time Series Analysis:**
Target Variable: Close (closing price for the given day)
Goal: To predict the closing price for the given day looking at past 60 days.Seasonality has been implemented for 14 days and 30 days periods.Moving averages, trend and seasonality are used to run regression time seriesanalysis.
* **4) Explore Arima model:** For Time series.


In [ ]:
#Import libraries here
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
#Import google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Go to the directory where the files are present
%cd /content/drive/My Drive/6211_stock_project

In [ ]:
#Load data from csv files to dataframes
df_large = pd.read_csv('large-cap-stocks-stocks-1y-test.csv', sep = '|')
df_small = pd.read_csv('small-cap-stocks-stocks-1y-test.csv', sep = '|')
df_mid = pd.read_csv('mid-cap-stocks-stocks-1y-test.csv', sep = '|')
df_mega = pd.read_csv('megacap-stocks-stocks-1y-test.csv', sep = '|')

In [ ]:
#viewing df_large dataset
df_large.head()

In [ ]:
#how many unique values are in df_small['Symbol']?
unique_symbols = df_small['Symbol'].unique()
print(unique_symbols)

In [ ]:
# Concatenate the 4 dataframes into a stocks dataframe
stocks = pd.concat([df_small, df_mid, df_large, df_mega],ignore_index=True)

In [ ]:
#Cheching for missing values in columns
stocks.isna().sum()

In [ ]:
# View details of stocks dataframe
stocks.info()

In [ ]:
stocks.duplicated().sum()

In [ ]:
# We see that the Symbol column in empty for some rows. Drop rows with null values from the stocks DataFrame.
stocks = stocks.dropna(subset=["Symbol"])
print(stocks.isna().mean()*100)

In [ ]:
# Create a copy of the stocks dataframe
stocks_copy = stocks.copy(deep = True)

In [ ]:
# Change the datatype for Date column to Date
try:
    stocks_copy['Date'] = pd.to_datetime(stocks_copy['Date'], format='%Y-%m-%d')
except Exception as e:
    print(f"Error: {e}")

stocks_copy.info()

In [ ]:
# View data
stocks_copy.head(10)

In [ ]:
# Inspect the shape of the stocks_copy dataframe
stocks_copy.shape

In [ ]:
stocks_copy.describe()

Exploratory Data Analysis(for EDA only one company(Microsoft) from the Mega capsize data is taken).

In [ ]:
# Filter rows where symbol is 'MSFT'
msft_df = stocks_copy[stocks_copy['Symbol'] == 'MSFT']

# Sort the filtered DataFrame by 'Date' column in ascending order
msft_df = msft_df.sort_values(by='Date', ascending=True)
msft_df.reset_index(drop=True, inplace=True)

# Display the sorted DataFrame
msft_df.head()

In [ ]:
# Define a function to calculate Exponential Moving Average
def exponential_moving_average(prices, period, weighting_factor=0.2):
    ema = np.zeros(len(prices))
    sma = np.mean(prices[:period])
    ema[period - 1] = sma
    for i in range(period, len(prices)):
        ema[i] = (prices[i] * weighting_factor) + (ema[i - 1] * (1 - weighting_factor))
    return ema

# Function to determine Relative Strength Index(RSI)
def RSI(data, window=14, adjust=False):
    delta = data['Close'].diff(1).dropna()
    loss = delta.copy()
    gains = delta.copy()

    gains[gains < 0] = 0
    loss[loss > 0] = 0

    gain_ewm = gains.ewm(com=window - 1, adjust=adjust).mean()
    loss_ewm = abs(loss.ewm(com=window - 1, adjust=adjust).mean())

    RS = gain_ewm / loss_ewm
    RSI = 100 - 100 / (1 + RS)

    return RSI

# Define function to calculate the Bollinger Bands
def BollingerBands(data, n):
    MA = data.Close.rolling(window=n).mean()
    SD = data.Close.rolling(window=n).std()
    data['MiddleBand'] = MA
    data['UpperBand'] = MA + (2 * SD)
    data['LowerBand'] = MA - (2 * SD)
    return data

# Define Gain and Loss functions
def gain(x):
    return ((x > 0) * x).sum()

def loss(x):
    return ((x < 0) * x).sum()

# Calculate money flow index(MFI)
def mfi(high, low, close, volume, n=14):
    typical_price = (high + low + close) / 3
    money_flow = typical_price * volume
    mf_sign = np.where(typical_price > typical_price.shift(1), 1, -1)
    signed_mf = money_flow * mf_sign

    # Calculate gain and loss using vectorized operations
    positive_mf = np.where(signed_mf > 0, signed_mf, 0)
    negative_mf = np.where(signed_mf < 0, -signed_mf, 0)

    mf_avg_gain = pd.Series(positive_mf).rolling(n, min_periods=1).sum()
    mf_avg_loss = pd.Series(negative_mf).rolling(n, min_periods=1).sum()

    return (100 - 100 / (1 + mf_avg_gain / mf_avg_loss)).to_numpy()

# Function is defined to return ATR values
def atr(high, low, close, n=14):
    tr = np.amax(np.vstack(((high - low).to_numpy(), (abs(high - close)).to_numpy(), (abs(low - close)).to_numpy())).T, axis=1)
    return pd.Series(tr).rolling(n).mean().to_numpy()

# Function to calculate ForceIndex
def ForceIndex(data, ndays):
    FI = pd.Series(data['Close'].diff(ndays) * data['Volume'], name = 'ForceIndex')
    data = data.join(FI)
    return data

# Function defined to return the data frame with Ease of Movement Indicator added to it
def EMV(data, ndays):
    dm = ((data['High'] + data['Low'])/2) - ((data['High'].shift(1) + data['Low'].shift(1))/2)
    br = (data['Volume'] / 100000000) / ((data['High'] - data['Low']))
    EMV = dm / br
    EMV_MA = pd.Series(EMV.rolling(ndays).mean(), name = 'EMV')
    data = data.join(EMV_MA)
    return data

In [ ]:
# Calculate Simple Moving Average for 14 days
msft_df['SMA14'] = msft_df.groupby('Symbol')['Close'].rolling(window=14).mean().reset_index(level=0, drop=True)

# Create a new column in the MSFT dataframe for Exponential Weighted Moving Average for 14 days
msft_df['EWMA'] = exponential_moving_average(msft_df['Close'], 14)

# Add a column for Relative Strength Index
msft_df["RSI"] = RSI(msft_df)

# Compute the Bollinger Bands for Google using the 10-day Moving average
n = 10
BolBands = BollingerBands(msft_df, n)

# Add a column with Money Flow Index
msft_df['MFI'] = mfi(msft_df['High'], msft_df['Low'], msft_df['Close'], msft_df['Volume'], 14)

# Add a column to for ATR
msft_df['ATR'] = atr(msft_df['High'], msft_df['Low'], msft_df['Close'], 14)

# Add a column to for ForceIndex
msft_df = ForceIndex(msft_df,1)

# Add a column to for EMV
msft_df = EMV(msft_df, 14)

msft_df.head(10)

In [ ]:
# Plotting the MSFT stock Price Series chart and Moving Averages below
plt.figure(figsize=(10,5))

# Set the title and axis labels
plt.title('Moving Average')
plt.xlabel('Date')
plt.ylabel('Price')

# Plot close price and moving averages
plt.plot(msft_df['Close'],lw=1, label='Close Price')
plt.plot(msft_df['SMA14'],'g',lw=1, label='14-day SMA')
plt.plot(msft_df['EWMA'],'r', lw=1, label='14-day EWMA')

# Add a legend to the axis
plt.legend()

plt.show()

In [ ]:
# Plotting the MSFT stock Price Series chart and Bollinger Bands
plt.figure(figsize=(10,7))

# Set the title and axis labels
plt.title('Bollinger Bands')
plt.xlabel('Date')
plt.ylabel('Price')

plt.plot(BolBands.Close,lw=1, label='Close Price')
plt.plot(msft_df['UpperBand'],'g',lw=1, label='Upper band')
plt.plot(msft_df['MiddleBand'],'r',lw=1, label='Middle band')
plt.plot(msft_df['LowerBand'],'g', lw=1, label='Lower band')

# Add a legend to the axis
plt.legend()

plt.show()

In [ ]:
# Plotting the Price Series chart and the RSI below
fig = plt.figure(figsize=(8, 10))

# Define position of 1st subplot
ax = fig.add_subplot(2, 1, 1)

# Set the title and axis labels
plt.title('Microsoft Price Chart')
plt.xlabel('Date')
plt.ylabel('Close Price')

plt.plot(msft_df['Close'], label='Close price')

# Add a legend to the axis
plt.legend()

# Define position of 2nd subplot
bx = fig.add_subplot(2, 1, 2)

# Set the title and axis labels
plt.title('Relative Strength Index')
plt.xlabel('Date')
plt.ylabel('RSI values')

plt.plot(msft_df['RSI'], 'm', label='RSI')

# Add a legend to the axis
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Plotting the Price Series chart and the MFI below
fig = plt.figure(figsize=(8, 10))

# Define position of 1st subplot
ax = fig.add_subplot(2, 1, 1)

# Set the title and axis labels
plt.title('Microsoft Price Chart')
plt.xlabel('Date')
plt.ylabel('Close Price')

plt.plot(msft_df['Close'], label='Close price')

# Add a legend to the axis
plt.legend()

# Define position of 2nd subplot
bx = fig.add_subplot(2, 1, 2)

# Set the title and axis labels
plt.title('Money flow index')
plt.xlabel('Date')
plt.ylabel('MFI values')

plt.plot(msft_df['MFI'], 'm', label='MFI')

# Add a legend to the axis
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Plotting the Price Series chart and the ATR below
fig = plt.figure(figsize=(8, 10))

# Define position of 1st subplot
ax = fig.add_subplot(2, 1, 1)

# Set the title and axis labels
plt.title('Microsoft Price Chart')
plt.xlabel('Date')
plt.ylabel('Close Price')

plt.plot(msft_df['Close'], label='Close price')

# Add a legend to the axis
plt.legend()

# Define position of 2nd subplot
bx = fig.add_subplot(2, 1, 2)

# Set the title and axis labels
plt.title('Average True Range')
plt.xlabel('Date')
plt.ylabel('ATR values')

plt.plot(msft_df['ATR'] , 'm', label='ATR')

# Add a legend to the axis
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Plotting the Price Series chart and the ForceIndex below
fig = plt.figure(figsize=(8, 10))

# Define position of 1st subplot
ax = fig.add_subplot(2, 1, 1)

# Set the title and axis labels
plt.title('Microsoft Price Chart')
plt.xlabel('Date')
plt.ylabel('Close Price')

# Plot the close price of the Apple
plt.plot(msft_df['Close'], label='Close price')

# Add a legend to the axis
plt.legend()

# Define position of 2nd subplot
bx = fig.add_subplot(2, 1, 2)

# Set the title and axis labels
plt.title('Force Index Chart')
plt.xlabel('Date')
plt.ylabel('ForceIndex values')

# Plot the ease of movement
plt.plot(msft_df['ForceIndex'], 'm', label='ForceIndex')

# Add a legend to the axis
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Plotting the Price Series chart and the Ease Of Movement below
fig = plt.figure(figsize=(8, 10))

# Define position of 1st subplot
ax = fig.add_subplot(2, 1, 1)

# Set the title and axis labels
plt.title('Microsoft Price Chart')
plt.xlabel('Date')
plt.ylabel('Close Price')

# Plot the close price of the Apple
plt.plot(msft_df['Close'], label='Close price')

# Add a legend to the axis
plt.legend()

# Define position of 2nd subplot
bx = fig.add_subplot(2, 1, 2)

# Set the title and axis labels
plt.title('Ease Of Movement Chart')
plt.xlabel('Date')
plt.ylabel('EMV values')

# Plot the ease of movement
plt.plot(msft_df['EMV'], 'm', label='EMV(14)')

# Add a legend to the axis
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:


# Sort DataFrame by company name and date
stocks_copy.sort_values(by=['Companyname', 'Date'], inplace=True)

# Compute the rolling mean with 7 days lag for each company
stocks_copy['Moving Avg Adj Close'] = stocks_copy.groupby('Companyname')['Adj Close'].transform(lambda x: x.rolling(window=7).mean())

# Plot the moving average for each company separately
for company_name, group in stocks_copy.groupby('Companyname'):
    plt.figure(figsize=(10, 6))
    plt.plot(group['Date'], group['Moving Avg Adj Close'])
    plt.xlabel('Date')
    plt.ylabel('Moving Average of Adjusted Close (7-Day Lag)')
    plt.title(f'Moving Average of Adjusted Close with 7-Day Lag for {company_name}')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:


# Sort DataFrame by company name and date
stocks_copy.sort_values(by=['Companyname', 'Date'], inplace=True)

# Compute the rolling mean with 7 days lag for each company
stocks_copy['Moving Avg Open'] = stocks_copy.groupby('Companyname')['Open'].transform(lambda x: x.rolling(window=7).mean())

# Plot the moving average for each company separately
for company_name, group in stocks_copy.groupby('companCompanynameyname'):
    plt.figure(figsize=(10, 6))
    plt.plot(group['Date'], group['Moving Avg Open'])
    plt.xlabel('Date')
    plt.ylabel('Moving Average of Open (7-Day Lag)')
    plt.title(f'Moving Average of Open with 7-Day Lag for {company_name}')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
# Drop rows with missing dates
stocks_copy.dropna(subset=['Date'], inplace=True)

# Sort DataFrame by company name and date
stocks_copy.sort_values(by=['companyname', 'Date'], inplace=True)

# Filter data for Microsoft
microsoft_data = stocks_copy[stocks_copy['companyname'] == 'Microsoft']

# Compute the rolling mean with 7 days lag for Microsoft
microsoft_data['Moving Avg Open'] = microsoft_data['Open'].rolling(window=7).mean()

# Plot the moving average for Microsoft
plt.figure(figsize=(10, 6))
plt.plot(microsoft_data['Date'], microsoft_data['Moving Avg Open'])
plt.xlabel('Date')
plt.ylabel('Moving Average of Open (7-Day Lag)')
plt.title('Moving Average of Open with 7-Day Lag for Microsoft')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Drop rows with missing dates
stocks_copy.dropna(subset=['Date'], inplace=True)

# Sort DataFrame by company name and date
stocks_copy.sort_values(by=['companyname', 'Date'], inplace=True)

# Filter data for Microsoft
microsoft_data = stocks_copy[stocks_copy['companyname'] == 'Microsoft']

# Compute the rolling mean with 7 days lag for Microsoft
microsoft_data['Moving Avg Open'] = microsoft_data['Open'].rolling(window=7).mean()

# Plot the moving average for Microsoft
plt.figure(figsize=(10, 6))
plt.plot(microsoft_data['Date'], microsoft_data['Moving Avg Open'])
plt.xlabel('Date')
plt.ylabel('Moving Average of Open (7-Day Lag)')
plt.title('Moving Average of Open with 7-Day Lag for Microsoft')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()


## **7. Conclusion:**

We have implemented multiple models to compare their performance to predict time-series stocks data. The findings so far with the models completed are:

a) Random Forest or ensemble models did not perform well with the data.
b) LSTM results show low mean square errors and predicted values following the actual values very closely.


**Next Steps:**

We are working on implementing traditional time series analysis on this data and evaluating the performance. We will next use moving averages, trend and seasonality to run regression time series analysis. We will also try to explore and run Arima model for analyzing this time series.